In [117]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import re

In [118]:
#Nos permite obtener el contenido de una web
def get_web_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Nos elimina el contenido de los scripts y estilos
    for script in soup(["script", "style"]):
        script.decompose()
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [119]:
# Nos divide el texto en chunks
def create_chunks(text, chunk_size=1000):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    print(f"Created {len(chunks)} chunks")
    return chunks

In [120]:
# Creamos el vector store
def create_vector_store(chunks):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(chunks)
    
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings.astype('float32'))
    
    return model, index

In [121]:
# Función de búsqueda se basa en la similitud de cosenos
def search(query, model, index, chunks, k=3):
    query_vector = model.encode([query])
    scores, indices = index.search(query_vector.astype('float32'), k)
    
    results = []
    for idx, score in zip(indices[0], scores[0]):
        results.append({
            'score': float(score),
            'content': chunks[idx][:200] + "..."
        })
    return results

In [122]:
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="Inserta tu api key"
)

def send_request(query):
    completion = client.chat.completions.create(
        model="nvidia/llama-3.1-nemotron-70b-instruct",
        messages=[{"role": "user", "content": query}],
        temperature=0.5,
        top_p=1,
        max_tokens=1024,
        stream=True
    )

    response_content = ""
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content

    return response_content

In [123]:
# Funcion para formatear en un prompt
def format_prompt(query, context):
    prompt = f"""
    In English always: based on the following context, answer the question: "{query}"
    
    {context}
    
    Answer:"""
    return prompt


In [124]:
if __name__ == "__main__":
    url = "https://github.com/ASKhem/proyect-ragx"
    text = get_web_content(url)
    
    # Creamos chunks
    chunks = create_chunks(text)
    
    # Creamos el vector store
    model, index = create_vector_store(chunks)
    
    # Realizamos la búsqueda
    query = "Según este repositorio como puedo participar como desarrollador?"
    results = search(query, model, index, chunks)
    context = results[0].get('content')
    
    # Formatea el resultado en un prompt
    prompt = format_prompt(query, context)
    
    # Enviamos el prompt a la API
    response = send_request(prompt)
    print(response)

Created 5 chunks
Based on the provided context, here is the answer to the question:

**"Según este repositorio, ¿cómo puedo participar como desarrollador?"**

**Respuesta:**
Para participar como desarrollador, sigue las **"Guías de contribución"** disponibles en el archivo **`CONTRIBUTING.md`**. Si tienes alguna duda adicional o necesitas contacto directo, puedes enviar un correo electrónico a **`khemwirtz@gmail.com`**. Para reportar problemas específicos, utiliza **`GitHub Issues`**.
